## GAN

GAN은 training 데이터 분포를 알아내 같은 분포로부터 새로운 데이터를 생성할 수 있는 모델

Generator : training 데이터와 유사한 가짜 이미지를 만들어 낸다.<br>
Discriminator : 이미지를 본 후 실제 이미지인지 generator부터 만들어진 이미지인지 출력하는 것이다.


## DCGAN

DAGAN은 GAN의 확장으로 generator와 discriminator 사이에서 각각 convolution layer와 convolutional-transpose(전치 합성곱)를 사용한다는 점에서 차이가 있다.
위 gan은 Unsupervised Representation Learning with Deep Convolution Generative Adversarial Networks 논문에서 처음 발표되었다.

Discriminator은 strided convolution layer, BN layer 그리고 leaky ReLU 활성화 함수로 구성되어 있다. input은 이미지이고 output은 input이 실제 데이터 분포에서 가져온 데이터일 확률이다.

Generator은 convolutional-transpose layer, BN layer 그리고 ReLU 활성화 함수로 구성된다.

## DCGAN 실습

https://github.com/YBIGTA/Deep_learning/blob/master/GAN/2017-08-12-DCGAN-korCeleb.markdown

### 1. 패키지 불러오기

In [17]:
from __future__ import print_function 
#%matplotlib inline 
import argparse 
import os 
import random 
import torch 
import torch.nn as nn 
import torch.nn.parallel 
import torch.backends.cudnn as cudnn 
import torch.optim as optim 
import torch.utils.data 
import torchvision.datasets as dset 
import torchvision.transforms as transforms 
import torchvision.utils as vutils 
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.animation as animation 
from IPython.display import HTML
import torchvision.datasets as dsets
from torch.autograd import Variable

In [4]:
# Set Random Seed 
seed = 999
print("Random seed : ", seed)
random.seed(seed)
torch.manual_seed(seed) # 파이토치 random seed 고정

Random seed :  999


### 2. 필요한 변수 설정

In [8]:
nz = 100 # 노이즈 벡터의 크기
nc = 3 # 채널의 수
ngf = 64 # generator 필터 조정
ndf = 64 # discriminator 필터 조정
niter = 200 # 에폭 수
lr = 0.0002
beta1 = 0.5

imageSize = 64 # 만들어지는 이미지의 크기
batchSize = 64 # 미니배치의 크기
outf = "result"

### 3. 데이터 불러오기

In [9]:
transform = transforms.Compose([
        transforms.Scale(64),
        transforms.ToTensor(),                     
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

train_dataset = dsets.CIFAR10(root='./data/', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size= batchSize, shuffle=True)

Files already downloaded and verified


In [12]:
for x,y in train_loader:
    print(x.shape, y.shape)
    break

torch.Size([64, 3, 64, 64]) torch.Size([64])


CIFAR10은 크기가 32x32 이미지이다. 그런데 dcgan은 일반적으로 64x64 이미지를 사용한다.

### 4. weight의 초기값 설정

In [13]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:         # Conv weight init
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:  # BatchNorm weight init
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

### 5. Generator 모델 정의

In [14]:
class _netG(nn.Module):
    def __init__(self):
        super(_netG, self).__init__()
        self.main = nn.Sequential(

            # 입력값은 Z이며 Transposed Convolution을 거칩니다.
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),

            # (ngf * 8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),

            # (ngf * 4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),

            # (ngf * 2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),

            # ngf x 32 x 32
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    def forward(self, input):
        output = self.main(input)
        return output

### 6. Discriminator 모델 정의

In [15]:
class _netD(nn.Module):
    def __init__(self):
        super(_netD, self).__init__()
        self.main = nn.Sequential(
            # (nc) x 64 x 64)
            nn.Conv2d(nc, ndf, 4,2,1,bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            # ndf x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),

            # (ndf * 2) x 16 x 16
            nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),

            # (ndf * 4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf*8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
    def forward(self, input):
        output = self.main(input)
        return output.view(-1, 1).squeeze(1)

In [18]:
netG = _netG()
netG.apply(weights_init)
#print(netG)

netD = _netD()
netD.apply(weights_init)
#print(netD)
criterion = nn.BCELoss()

input = torch.FloatTensor(batchSize, 3, imageSize, imageSize)
noise = torch.FloatTensor(batchSize, nz, 1, 1)
fixed_noise = torch.FloatTensor(batchSize, nz, 1, 1).normal_(0, 1)
fixed_noise = Variable(fixed_noise)

label = torch.FloatTensor(batchSize)
real_label = 1
fake_label = 0

_netG(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)
_net

In [19]:
# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [26]:
epochNum=10
loss_D = []
loss_G = []
score_G1 = []
score_G2 = []
score_D = []
for epoch in range(epochNum):
    for i, data in enumerate(train_loader):
        # train with real
        netD.zero_grad()
        real_cpu, _ = data
        batch_size = real_cpu.size(0)

        input.resize_as_(real_cpu).copy_(real_cpu)
        label.resize_(batch_size).fill_(real_label)

        inputv = Variable(input)
        labelv = Variable(label)

        output = netD(inputv)
        errD_real = criterion(output, labelv)
        errD_real.backward()
        D_x = output.data.mean()

        # train with fake
        noise.resize_(batch_size, nz, 1, 1).normal_(0, 1)
        noisev = Variable(noise)
        fake = netG(noisev)
        labelv = Variable(label.fill_(fake_label))
        output = netD(fake.detach())
        errD_fake = criterion(output, labelv)
        errD_fake.backward()
        D_G_z1 = output.data.mean()

        errD = errD_real + errD_fake
        optimizerD.step()

        netG.zero_grad()
        labelv = Variable(label.fill_(real_label))
        output = netD(fake)

        errG = criterion(output, labelv)
        errG.backward()
        D_G_z2 = output.data.mean()
        optimizerG.step()
        if ((i+1) % 100 == 0):
            print(i, "step")
            fake = netG(fixed_noise)
            vutils.save_image(fake.data,
            '%s/fake_samples_epoch_%s.png' % (outf, str(epoch)+" "+str(i+1)),
            normalize=True)
    vutils.save_image(real_cpu,
            '%s/real_samples.png' % outf,
            normalize=True)
    fake = netG(fixed_noise)
    vutils.save_image(fake.data,
            '%s/fake_samples_epoch_%s.png' % (outf, epoch),
            normalize=True)
    result_dict = {"loss_D":loss_D,"loss_G":loss_G,"score_D":score_D,"score_G1":score_G1,"score_G2":score_G2}

    # do checkpointing
    torch.save(netG.state_dict(), '%s/netG.pth' % (outf))
    torch.save(netD.state_dict(), '%s/netD.pth' % (outf))

99 step
199 step
299 step
399 step
499 step
599 step
699 step
99 step
199 step
299 step
399 step
499 step
599 step
699 step
99 step
199 step
299 step
399 step
499 step
599 step
699 step
99 step
199 step
299 step
399 step
499 step
599 step
699 step
99 step
199 step
299 step
399 step
499 step
599 step
699 step
99 step
199 step
299 step
399 step
499 step
599 step
699 step
99 step
199 step
299 step
399 step
499 step
599 step
699 step
99 step
199 step
299 step
399 step
499 step
599 step
699 step
99 step
199 step
299 step
399 step
499 step
599 step
699 step
99 step
199 step
299 step
399 step
499 step
599 step
699 step
